In [44]:
!pip install torchsummary

In [1]:
import torch
from torchsummary import summary
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# import visdom

from torchvision import datasets, transforms

In [2]:
seed = 1

momentum = 0.5

batch_size = 128
test_batch_size = 128

no_cuda = False

## Model

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
            )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),
    
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )

        self.model = nn.Sequential(
            conv_bn(  3,  32, 2), 
            conv_dw( 32,  64, 1),
            conv_dw( 64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AvgPool2d(3),
        )
        self.fc = nn.Linear(1024, 9)

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

In [4]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [5]:
model = Net().to(device)

In [9]:
summary(model, input_size=(3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             864
       BatchNorm2d-2           [-1, 32, 64, 64]              64
              ReLU-3           [-1, 32, 64, 64]               0
            Conv2d-4           [-1, 32, 64, 64]             288
       BatchNorm2d-5           [-1, 32, 64, 64]              64
              ReLU-6           [-1, 32, 64, 64]               0
            Conv2d-7           [-1, 64, 64, 64]           2,048
       BatchNorm2d-8           [-1, 64, 64, 64]             128
              ReLU-9           [-1, 64, 64, 64]               0
           Conv2d-10           [-1, 64, 32, 32]             576
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 32, 32]           8,192
      BatchNorm2d-14          [-1, 128,

In [11]:
train_dir = 'dataset/tomato/'
# test_dir = 'dataset/mnist/testing/'

In [12]:
train_dataset = datasets.ImageFolder(root=train_dir,
                                    transform=transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.1307,), (0.3081,)),
                                    ]))
# test_dataset = datasets.ImageFolder(root=test_dir,
#                                     transform=transforms.Compose([
#                                         transforms.ToTensor(),
#                                         transforms.Normalize((0.1307,), (0.3081,))
#                                     ]))

In [115]:
# train_loader = torch.utils.data.DataLoader(train_dataset,
#                                           batch_size = batch_size,
#                                           shuffle = True)
# test_loader = torch.utils.data.DataLoader(test_dataset,
#                                           batch_size = batch_size,
#                                           shuffle = False)

In [13]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = len(train_dataset.imgs))

In [14]:
for image, label in train_loader:
    image = image
    label = label
    print(image.shape)
    print(label.shape)

torch.Size([17063, 3, 128, 128])
torch.Size([17063])


In [15]:
data = image.view(-1,128*128*3)
target = label.view(-1,1)

In [16]:
dataset_x = data.numpy()
dataset_y = target.numpy().astype(int)

In [17]:
import numpy as np
np.save('./dataset/image.npy', dataset_x)
np.save('./dataset/label.npy', dataset_y)

In [6]:
import numpy as np
dataset_x = np.load('./dataset/image.npy')
dataset_y = np.load('./dataset/label.npy')

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(dataset_x, dataset_y, test_size=0.1, random_state=1,stratify=dataset_y)


In [8]:
X_train.shape

(15356, 49152)

In [10]:
y_train.shape

(15356, 1)

In [15]:
y_train[0:100,0]

array([7, 7, 6, 6, 6, 6, 2, 6, 6, 6, 7, 6, 6, 6, 2, 7, 6, 6, 7, 6, 7, 7,
       7, 7, 7, 6, 6, 6, 6, 4, 6, 6, 6, 7, 2, 2, 7, 6, 6, 6, 6, 2, 6, 6,
       7, 6, 6, 7, 7, 2, 2, 7, 6, 6, 2, 6, 6, 2, 2, 7, 6, 2, 6, 1, 7, 2,
       2, 4, 6, 6, 1, 6, 6, 6, 6, 4, 6, 6, 7, 6, 6, 6, 7, 7, 6, 6, 6, 6,
       2, 6, 7, 7, 1, 7, 6, 6, 6, 7, 6, 7])

In [9]:
X_train = torch.FloatTensor(X_train).view([-1,3,128,128])
y_train = torch.LongTensor(y_train).view([-1])
X_val = torch.FloatTensor(X_val).view([-1,3,128,128])
y_val = torch.LongTensor(y_val).view([-1])


In [10]:
y_val.bincount(), y_train.bincount()

(tensor([ 23,  77, 206,   9,  42,   5, 996, 335,  14]),
 tensor([ 206,  689, 1855,   81,  384,   44, 8959, 3013,  125]))

In [11]:
X_train.shape, y_train.shape

(torch.Size([15356, 3, 128, 128]), torch.Size([15356]))

In [12]:
trainDataset = torch.utils.data.TensorDataset(X_train, y_train)
validDataset = torch.utils.data.TensorDataset(X_val, y_val)

In [13]:
Train_loader = torch.utils.data.DataLoader(trainDataset,
                                          batch_size = batch_size,
                                          shuffle = False)
Valid_loader = torch.utils.data.DataLoader(validDataset,
                                          batch_size = len(X_val),
                                          shuffle = False)

In [14]:
from datetime import datetime

In [15]:
print(datetime.now())

2019-11-15 02:15:02.550798


In [16]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [18]:
import torchvision
from torch.utils.tensorboard import SummaryWriter

In [20]:
lr = 0.002
epochs = 100

In [21]:
model = Net().to(device)

In [22]:
optimizer = optim.SGD(model.parameters(),lr=lr,momentum=momentum)
loss_func = nn.CrossEntropyLoss()

In [23]:
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=1, verbose=True)

In [24]:
images, labels = next(iter(Train_loader))
writer = SummaryWriter(comment='Model')
writer.add_graph(Net(), input_to_model=images)
writer.close()

In [25]:
writer_t = SummaryWriter(comment=' Train / epoch : {} / learning_rate : {}'.format(epochs,lr))
writer_v = SummaryWriter(comment=' Val / epoch : {} / learning_rate : {}'.format(epochs,lr))

correct_train = 0
total_train = 0
for i in range(epochs):
    start_time = datetime.now()
    for j, (data, label) in enumerate(Train_loader):
        x = data.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
#         print(output.max().shape,y_.shape)
        _, predicted = torch.max(output.data, 1)
        correct_train += (predicted == y_).sum().item()
        total_train += y_.size(0)
        accuracy_train = (100 * correct_train / total_train)
#         print('a')
        
        if (j+1) % 50 == 0:
            print('Epoch:{} / batch:{} / loss {:.2f}'.format(i+1,j+1, loss))
    with torch.no_grad():
        for val_x, val_y in Valid_loader:
            val_x = val_x.to(device)
            val_y = val_y.to(device)
            val_output = model.forward(val_x)
            v_loss = loss_func(val_output, val_y)
            _, predicted_val = torch.max(val_output.data, 1)
            accuracy_val = ( 100 * (predicted_val == val_y).sum().item() / len(val_x))
    end_time = datetime.now()    
    print('Epoch {} / time {} / loss {:.2f} / val_loss {:.2f} / acc {:.2f} / val_acc {:.2f}'.format(i+1, end_time - start_time, loss, v_loss, accuracy_train, accuracy_val))
    scheduler.step(v_loss, i)
#     if i == 0:
#       grid = torchvision.utils.make_grid(data)
#       writer.add_image('images',grid, i)
    writer_t.add_scalar('Loss', loss, i+1)
    writer_v.add_scalar('Loss', v_loss, i+1)
    writer_t.add_scalar('Acc', accuracy_train, i+1)
    writer_v.add_scalar('Acc', accuracy_val, i+1)

writer_t.close()
writer_v.close()


Epoch:1 / batch:50 / loss 1.26
Epoch:1 / batch:100 / loss 1.13
Epoch 1 / time 0:00:38.467473 / loss 1.16 / val_loss 1.21 / acc 57.09 / val_acc 58.35
Epoch:2 / batch:50 / loss 1.17
Epoch:2 / batch:100 / loss 1.08
Epoch 2 / time 0:00:37.614701 / loss 1.16 / val_loss 1.17 / acc 57.73 / val_acc 58.88
Epoch:3 / batch:50 / loss 1.10
Epoch:3 / batch:100 / loss 1.00
Epoch 3 / time 0:00:37.644803 / loss 1.09 / val_loss 1.06 / acc 58.50 / val_acc 64.32
Epoch:4 / batch:50 / loss 0.88
Epoch:4 / batch:100 / loss 0.87
Epoch 4 / time 0:00:37.618618 / loss 0.93 / val_loss 0.91 / acc 60.92 / val_acc 71.94
Epoch:5 / batch:50 / loss 0.80
Epoch:5 / batch:100 / loss 0.79
Epoch 5 / time 0:00:37.619600 / loss 0.85 / val_loss 0.86 / acc 63.34 / val_acc 74.40
Epoch:6 / batch:50 / loss 0.68
Epoch:6 / batch:100 / loss 0.78
Epoch 6 / time 0:00:37.666696 / loss 0.79 / val_loss 0.81 / acc 65.23 / val_acc 76.80
Epoch:7 / batch:50 / loss 0.67
Epoch:7 / batch:100 / loss 0.79
Epoch:9 / batch:50 / loss 0.57
Epoch:9 / ba

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        x = image.to(device)
        y_ = label.to(device)
        
        output = model.forward(x)
        _, output_index = torch.max(output,1)
        
        total += label.size(0)
        correct += (output_index == y_).sum().float()
        
    print("Accuracy of Test Data: {}".format(100*correct/total))

In [26]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'